In [1]:
import sys
sys.path.append("../")

import sqlite3
import pandas as pd
from omegaconf import OmegaConf
from copy import deepcopy
import torch
import numpy as np
from src.utils.setup_model import setup
from tqdm import tqdm
# Import sigmoid function
from torch.nn.functional import sigmoid
try:
    OmegaConf.register_new_resolver("eval", eval)
except ValueError:
    pass

In [2]:
def get_predictions(problem_type, table, db_path, device="cuda"):

    res_conn = sqlite3.connect(f'../out/{problem_type}/results.db')
    res_cur = res_conn.cursor()

    runs = pd.read_sql("SELECT * FROM " + table, res_conn)

    config_path = f"../out/{problem_type}/24/rotary_transformer/.hydra/config.yaml"

    omega_conf = OmegaConf.load(config_path)
    omega_conf.local.db_path = "../data/features.db"

    base_config = OmegaConf.to_container(omega_conf, resolve=True)

    base_config["logger"]["args"]["mode"] = "disabled"

    # Change device in config
    base_config["shared"]["device"] = device

    base_config["data"]["dataset"]["train"]["args"]["splits"]

    base_config["train"]["load_checkpoint"] = False
    base_config["cross_validation"]["use_checkpoints"] = False

    # The db should be in the same directory as the main db
    pred_conn = sqlite3.connect(db_path)
    pred_cur = pred_conn.cursor()

    # Now we need to create a table for the predictions
    # Drop if it exists
    pred_cur.execute(f"DROP TABLE IF EXISTS {problem_type}_{table}_predictions")
    pred_cur.execute(f"""CREATE TABLE {problem_type}_{table}_predictions 
                (run_id INTEGER, 
                harpnum INT,
                start TEXT,
                end TEXT,
                activity_in_obs REAL,
                flare_id INT,
                cme_id INT,
                cme_type INT,
                has_cme_flare_above_threshold INT,
                has_cme_flare_below_threshold INT,
                has_cme_no_flare INT,
                has_flare_above_threshold INT,
                has_flare_below_threshold INT,
                pred REAL, 
                target REAL,
                PRIMARY KEY (run_id, harpnum, end)
                )"""
                )


    for idx, run in tqdm(runs.iterrows()):
        config = deepcopy(base_config)

        # Set the splits correctly

        config["data"]["dataset"]["train"]["args"]["splits"] = pd.eval(run["train_splits"])
        config["data"]["dataset"]["val"]["args"]["splits"] = pd.eval(run["val_splits"])

        checkpoint_dir = run["checkpoint_dir"]

        # Break checkpoint_dir path where the subfolder out is 
        checkpoint_dir = "../out" + checkpoint_dir.split("out")[1]

        best_epoch = run["best_epoch"]
        checkpoint_path = checkpoint_dir + f"/checkpoint_{best_epoch}.pth"

        # Read the checkpoint

        checkpoint = torch.load(checkpoint_path, map_location=device)

        # Now load all the bits
        model_parts = setup(config)

        # Now need to load the checkpoint to the model

        model = model_parts["model"]
        model.load_state_dict(checkpoint["model_state_dict"])

        model.eval()

        # Now we can make predictions for the whole thing.
        # We'll store these in a new database.

        # Now we can go through the validation dataloader
        val_dataloader = model_parts["dataloaders"]["val"]

        run_id = run["run_id"]
        results = []
        for batch in tqdm(val_dataloader):
            data, label, metadata = batch
            data = data.to(device)
            label = label.to(device)

            with torch.no_grad():
                pred = sigmoid(model(data))
                pred = pred.cpu().numpy()
                label = label.cpu().numpy()

            cme_ids = metadata.get("cme_id", None)
            flare_ids = metadata.get("flare_id", None)
            cme_types = metadata.get("cme_type", None)
            has_cme_flare_above_thresholds = metadata.get("has_cme_flare_above_threshold", None)
            has_cme_flare_below_thresholds = metadata.get("has_cme_flare_below_threshold", None)
            has_cme_no_flares = metadata.get("has_cme_no_flare", None)
            has_flare_above_thresholds = metadata.get("has_flare_above_threshold", None)
            has_flare_below_thresholds = metadata.get("has_flare_below_threshold", None)


            for i in range(len(pred)):
                harpnum = int(metadata["harpnum"][i].cpu().numpy())
                start = metadata["start_date"][i]
                end = metadata["end_date"][i]
                if cme_ids is not None:
                    cme_id = cme_ids[i].cpu().numpy()
                    cme_id = int(cme_id) if not np.isnan(cme_id) else None
                else:
                    cme_id = None
                
                if flare_ids is not None:
                    flare_id = flare_ids[i].cpu().numpy()
                    flare_id = int(flare_id) if not np.isnan(flare_id) else None
                else:
                    flare_id = None

                if cme_types is not None:
                    cme_type = cme_types[i].cpu().numpy()
                    cme_type = int(cme_type) if not np.isnan(cme_type) else None
                else:
                    cme_type = None

                if has_cme_flare_above_thresholds is not None:
                    has_cme_flare_above_threshold = has_cme_flare_above_thresholds[i].cpu().numpy()
                    has_cme_flare_above_threshold = int(has_cme_flare_above_threshold)
                else:
                    has_cme_flare_above_threshold = None

                if has_cme_flare_below_thresholds is not None:
                    has_cme_flare_below_threshold = has_cme_flare_below_thresholds[i].cpu().numpy()
                    has_cme_flare_below_threshold = int(has_cme_flare_below_threshold)
                else:
                    has_cme_flare_below_threshold = None

                if has_cme_no_flares is not None:
                    has_cme_no_flare = has_cme_no_flares[i].cpu().numpy()
                    has_cme_no_flare = int(has_cme_no_flare)
                else:
                    has_cme_no_flare = None

                if has_flare_above_thresholds is not None:
                    has_flare_above_threshold = has_flare_above_thresholds[i].cpu().numpy()
                    has_flare_above_threshold = int(has_flare_above_threshold)
                else:
                    has_flare_above_threshold = None

                if has_flare_below_thresholds is not None:
                    has_flare_below_threshold = has_flare_below_thresholds[i].cpu().numpy()
                    has_flare_below_threshold = int(has_flare_below_threshold)
                else:
                    has_flare_below_threshold = None

                activity_in_obs = int(metadata["activity_in_obs"][i].cpu().numpy())
                spred = float(pred[i][0])
                slabel = int(label[i])

                results.append((run_id, harpnum, start, end, activity_in_obs, flare_id, cme_id, cme_type, spred, slabel, 
                                has_cme_flare_above_threshold, has_cme_flare_below_threshold, has_cme_no_flare, 
                                has_flare_above_threshold, has_flare_below_threshold))

        # Insert into databasae

        pred_cur.executemany(f"""INSERT INTO {problem_type}_{table}_predictions
                                    (run_id, harpnum, start, end, activity_in_obs, flare_id, cme_id, cme_type, pred, target, 
                                    has_cme_flare_above_threshold, has_cme_flare_below_threshold, has_cme_no_flare,
                                    has_flare_above_threshold, has_flare_below_threshold)
                                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""", results)

        pred_conn.commit()

In [3]:
# Override device as appropriate
get_predictions("flare_forecast", "rotary_transformer_24", "../out/main_predictions.db", device="cuda")

0it [00:00, ?it/s]

Loading Datasets
----------------
Loading train dataset
INITIALIZING CONNECTION
COPYING DATASET TABLE
CREATING TEMPORARY TABLE
NORMALIZING FEATURES
APPLYING PADDING STRATEGY
CLOSE CONNECTION
Not using keywords:  ['Bhis1d']
{'db_path': '../data/tmp/2a48e09c-abbc-4f70-91e8-8e9f2b56dcc2.db', 'T': 24, 'L': 24, 'S': 0.2, 'B': 0, 'C': 0.4, 'MIN_FLARE_CLASS': 30, 'ALLOW_OVERLAPS': 1, 'padding_strategy': 'zero', 'keywords': 'default', 'not_keywords': ['Bhis1d'], 'use_PCA': False, 'n_PCA_components': 0, 'mode': 'train', 'splits': [2, 3, 4, 5, 6, 7, 8, 9], 'noise_level': 0.05, 'use_only_active': True}
NOT USING PCA
Loading val dataset
INITIALIZING CONNECTION
COPYING DATASET TABLE
CREATING TEMPORARY TABLE
NORMALIZING FEATURES
APPLYING PADDING STRATEGY
CLOSE CONNECTION
Not using keywords:  ['Bhis1d']
{'db_path': '../data/tmp/2a48e09c-abbc-4f70-91e8-8e9f2b56dcc2.db', 'T': 24, 'L': 24, 'S': 0.2, 'B': 0, 'C': 0.4, 'MIN_FLARE_CLASS': 30, 'ALLOW_OVERLAPS': 1, 'padding_strategy': 'zero', 'keywords': 'de

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


{'base_lr': 5e-06, 'max_lr': 2e-05, 'step_size_up': 5796, 'cycle_momentum': False, 'mode': 'triangular2'}


100%|██████████| 1253/1253 [00:46<00:00, 26.79it/s]
1it [01:25, 85.68s/it]

...

100%|██████████| 1126/1126 [00:42<00:00, 26.72it/s]
8it [10:45, 78.65s/it]

Loading Datasets
----------------
Loading train dataset
INITIALIZING CONNECTION
COPYING DATASET TABLE
CREATING TEMPORARY TABLE
NORMALIZING FEATURES
APPLYING PADDING STRATEGY
CLOSE CONNECTION
Not using keywords:  ['Bhis1d']
{'db_path': '../data/tmp/3a40c08d-9c42-4141-a59e-ffb6e28fcad6.db', 'T': 24, 'L': 24, 'S': 0.2, 'B': 0, 'C': 0.4, 'MIN_FLARE_CLASS': 30, 'ALLOW_OVERLAPS': 1, 'padding_strategy': 'zero', 'keywords': 'default', 'not_keywords': ['Bhis1d'], 'use_PCA': False, 'n_PCA_components': 0, 'mode': 'train', 'splits': [0, 1, 2, 3, 4, 5, 6, 7], 'noise_level': 0.05, 'use_only_active': True}
NOT USING PCA
Loading val dataset
INITIALIZING CONNECTION
COPYING DATASET TABLE
CREATING TEMPORARY TABLE
NORMALIZING FEATURES
APPLYING PADDING STRATEGY
CLOSE CONNECTION
Not using keywords:  ['Bhis1d']
{'db_path': '../data/tmp/3a40c08d-9c42-4141-a59e-ffb6e28fcad6.db', 'T': 24, 'L': 24, 'S': 0.2, 'B': 0, 'C': 0.4, 'MIN_FLARE_CLASS': 30, 'ALLOW_OVERLAPS': 1, 'padding_strategy': 'zero', 'keywords': 'de

100%|██████████| 1166/1166 [00:43<00:00, 26.74it/s]
9it [12:05, 78.82s/it]

Loading Datasets
----------------
Loading train dataset
INITIALIZING CONNECTION
COPYING DATASET TABLE
CREATING TEMPORARY TABLE
NORMALIZING FEATURES
APPLYING PADDING STRATEGY
CLOSE CONNECTION
Not using keywords:  ['Bhis1d']
{'db_path': '../data/tmp/9d775af1-c56d-4fe6-a32c-1424ac91ed63.db', 'T': 24, 'L': 24, 'S': 0.2, 'B': 0, 'C': 0.4, 'MIN_FLARE_CLASS': 30, 'ALLOW_OVERLAPS': 1, 'padding_strategy': 'zero', 'keywords': 'default', 'not_keywords': ['Bhis1d'], 'use_PCA': False, 'n_PCA_components': 0, 'mode': 'train', 'splits': [1, 2, 3, 4, 5, 6, 7, 8], 'noise_level': 0.05, 'use_only_active': True}
NOT USING PCA
Loading val dataset
INITIALIZING CONNECTION
COPYING DATASET TABLE
CREATING TEMPORARY TABLE
NORMALIZING FEATURES
APPLYING PADDING STRATEGY
CLOSE CONNECTION
Not using keywords:  ['Bhis1d']
{'db_path': '../data/tmp/9d775af1-c56d-4fe6-a32c-1424ac91ed63.db', 'T': 24, 'L': 24, 'S': 0.2, 'B': 0, 'C': 0.4, 'MIN_FLARE_CLASS': 30, 'ALLOW_OVERLAPS': 1, 'padding_strategy': 'zero', 'keywords': 'de

100%|██████████| 1221/1221 [00:45<00:00, 26.68it/s]
10it [13:25, 80.55s/it]


In [4]:
# Override device as appropriate
get_predictions("cme_forecast", "rotary_transformer_24", "../out/main_predictions.db", device="cuda")

0it [00:00, ?it/s]

Loading Datasets
----------------
Loading train dataset
INITIALIZING CONNECTION
COPYING DATASET TABLE
CREATING TEMPORARY TABLE
NORMALIZING FEATURES
APPLYING PADDING STRATEGY
CLOSE CONNECTION
Not using keywords:  ['Bhis1d']
{'db_path': '../data/tmp/eeb265c0-dca4-412d-b589-c945d9542af3.db', 'T': 24, 'L': 24, 'S': 0.2, 'B': 0, 'C': 0.4, 'MIN_FLARE_CLASS': 30, 'ALLOW_OVERLAPS': 1, 'padding_strategy': 'zero', 'keywords': 'default', 'not_keywords': ['Bhis1d'], 'use_PCA': False, 'n_PCA_components': 0, 'mode': 'train', 'splits': [2, 3, 4, 5, 6, 7, 8, 9], 'noise_level': 0.05, 'use_only_active': True}
NOT USING PCA
Loading val dataset
INITIALIZING CONNECTION
COPYING DATASET TABLE
CREATING TEMPORARY TABLE
NORMALIZING FEATURES
APPLYING PADDING STRATEGY
CLOSE CONNECTION
Not using keywords:  ['Bhis1d']
{'db_path': '../data/tmp/eeb265c0-dca4-412d-b589-c945d9542af3.db', 'T': 24, 'L': 24, 'S': 0.2, 'B': 0, 'C': 0.4, 'MIN_FLARE_CLASS': 30, 'ALLOW_OVERLAPS': 1, 'padding_strategy': 'zero', 'keywords': 'de

100%|██████████| 1253/1253 [00:47<00:00, 26.29it/s]
1it [01:34, 94.91s/it]

Loading Datasets
----------------
Loading train dataset
INITIALIZING CONNECTION
COPYING DATASET TABLE
CREATING TEMPORARY TABLE
NORMALIZING FEATURES
APPLYING PADDING STRATEGY
CLOSE CONNECTION
Not using keywords:  ['Bhis1d']
{'db_path': '../data/tmp/64a75516-c899-4839-88e8-1de1d745ecbc.db', 'T': 24, 'L': 24, 'S': 0.2, 'B': 0, 'C': 0.4, 'MIN_FLARE_CLASS': 30, 'ALLOW_OVERLAPS': 1, 'padding_strategy': 'zero', 'keywords': 'default', 'not_keywords': ['Bhis1d'], 'use_PCA': False, 'n_PCA_components': 0, 'mode': 'train', 'splits': [0, 3, 4, 5, 6, 7, 8, 9], 'noise_level': 0.05, 'use_only_active': True}
NOT USING PCA
Loading val dataset
INITIALIZING CONNECTION
COPYING DATASET TABLE
CREATING TEMPORARY TABLE
NORMALIZING FEATURES
APPLYING PADDING STRATEGY
CLOSE CONNECTION
Not using keywords:  ['Bhis1d']
{'db_path': '../data/tmp/64a75516-c899-4839-88e8-1de1d745ecbc.db', 'T': 24, 'L': 24, 'S': 0.2, 'B': 0, 'C': 0.4, 'MIN_FLARE_CLASS': 30, 'ALLOW_OVERLAPS': 1, 'padding_strategy': 'zero', 'keywords': 'de

100%|██████████| 1303/1303 [00:49<00:00, 26.38it/s]
2it [03:11, 96.08s/it]

...

100%|██████████| 1126/1126 [00:42<00:00, 26.23it/s]
8it [12:41, 93.41s/it]

Loading Datasets
----------------
Loading train dataset
INITIALIZING CONNECTION
COPYING DATASET TABLE
CREATING TEMPORARY TABLE
NORMALIZING FEATURES
APPLYING PADDING STRATEGY
CLOSE CONNECTION
Not using keywords:  ['Bhis1d']
{'db_path': '../data/tmp/cb1ef273-91f7-4f1c-849b-a327bdaa8b68.db', 'T': 24, 'L': 24, 'S': 0.2, 'B': 0, 'C': 0.4, 'MIN_FLARE_CLASS': 30, 'ALLOW_OVERLAPS': 1, 'padding_strategy': 'zero', 'keywords': 'default', 'not_keywords': ['Bhis1d'], 'use_PCA': False, 'n_PCA_components': 0, 'mode': 'train', 'splits': [0, 1, 2, 3, 4, 5, 6, 7], 'noise_level': 0.05, 'use_only_active': True}
NOT USING PCA
Loading val dataset
INITIALIZING CONNECTION
COPYING DATASET TABLE
CREATING TEMPORARY TABLE
NORMALIZING FEATURES
APPLYING PADDING STRATEGY
CLOSE CONNECTION
Not using keywords:  ['Bhis1d']
{'db_path': '../data/tmp/cb1ef273-91f7-4f1c-849b-a327bdaa8b68.db', 'T': 24, 'L': 24, 'S': 0.2, 'B': 0, 'C': 0.4, 'MIN_FLARE_CLASS': 30, 'ALLOW_OVERLAPS': 1, 'padding_strategy': 'zero', 'keywords': 'de

100%|██████████| 1166/1166 [00:44<00:00, 26.07it/s]
9it [14:15, 93.45s/it]

Loading Datasets
----------------
Loading train dataset
INITIALIZING CONNECTION
COPYING DATASET TABLE
CREATING TEMPORARY TABLE
NORMALIZING FEATURES
APPLYING PADDING STRATEGY
CLOSE CONNECTION
Not using keywords:  ['Bhis1d']
{'db_path': '../data/tmp/48e09e6e-033e-42a3-ada1-3f393398d0a0.db', 'T': 24, 'L': 24, 'S': 0.2, 'B': 0, 'C': 0.4, 'MIN_FLARE_CLASS': 30, 'ALLOW_OVERLAPS': 1, 'padding_strategy': 'zero', 'keywords': 'default', 'not_keywords': ['Bhis1d'], 'use_PCA': False, 'n_PCA_components': 0, 'mode': 'train', 'splits': [1, 2, 3, 4, 5, 6, 7, 8], 'noise_level': 0.05, 'use_only_active': True}
NOT USING PCA
Loading val dataset
INITIALIZING CONNECTION
COPYING DATASET TABLE
CREATING TEMPORARY TABLE
NORMALIZING FEATURES
APPLYING PADDING STRATEGY
CLOSE CONNECTION
Not using keywords:  ['Bhis1d']
{'db_path': '../data/tmp/48e09e6e-033e-42a3-ada1-3f393398d0a0.db', 'T': 24, 'L': 24, 'S': 0.2, 'B': 0, 'C': 0.4, 'MIN_FLARE_CLASS': 30, 'ALLOW_OVERLAPS': 1, 'padding_strategy': 'zero', 'keywords': 'de

100%|██████████| 1221/1221 [00:46<00:00, 26.24it/s]
10it [15:50, 95.03s/it]


In [5]:
# Override device as appropriate
get_predictions("flare_cme_assoc", "rotary_transformer_24", "../out/main_predictions.db", device="cuda")

0it [00:00, ?it/s]

Loading Datasets
----------------
Loading train dataset
INITIALIZING CONNECTION
COPYING DATASET TABLE
CREATING TEMPORARY TABLE
NORMALIZING FEATURES
APPLYING PADDING STRATEGY
CLOSE CONNECTION
Not using keywords:  ['Bhis1d']
{'db_path': '../data/tmp/5e2f0baa-f22d-4f70-bc15-6e4145b0841f.db', 'T': 24, 'L': 24, 'S': 0.2, 'B': 0, 'C': 0.4, 'MIN_FLARE_CLASS': 30, 'ALLOW_OVERLAPS': 1, 'padding_strategy': 'zero', 'keywords': 'default', 'not_keywords': ['Bhis1d'], 'use_PCA': False, 'n_PCA_components': 0, 'mode': 'train', 'splits': [2, 3, 4, 5, 6, 7, 8, 9], 'noise_level': 0.05, 'use_only_active': True}
NOT USING PCA
Loading val dataset
INITIALIZING CONNECTION
COPYING DATASET TABLE
CREATING TEMPORARY TABLE
NORMALIZING FEATURES
APPLYING PADDING STRATEGY
CLOSE CONNECTION
Not using keywords:  ['Bhis1d']
{'db_path': '../data/tmp/5e2f0baa-f22d-4f70-bc15-6e4145b0841f.db', 'T': 24, 'L': 24, 'S': 0.2, 'B': 0, 'C': 0.4, 'MIN_FLARE_CLASS': 30, 'ALLOW_OVERLAPS': 1, 'padding_strategy': 'zero', 'keywords': 'de

100%|██████████| 25/25 [00:01<00:00, 19.71it/s]
1it [00:32, 32.35s/it]

Loading Datasets
----------------
Loading train dataset
INITIALIZING CONNECTION
COPYING DATASET TABLE
CREATING TEMPORARY TABLE
NORMALIZING FEATURES
APPLYING PADDING STRATEGY
CLOSE CONNECTION
Not using keywords:  ['Bhis1d']
{'db_path': '../data/tmp/67de8e44-4dd3-4ebc-b993-2896b1c8dbe1.db', 'T': 24, 'L': 24, 'S': 0.2, 'B': 0, 'C': 0.4, 'MIN_FLARE_CLASS': 30, 'ALLOW_OVERLAPS': 1, 'padding_strategy': 'zero', 'keywords': 'default', 'not_keywords': ['Bhis1d'], 'use_PCA': False, 'n_PCA_components': 0, 'mode': 'train', 'splits': [0, 3, 4, 5, 6, 7, 8, 9], 'noise_level': 0.05, 'use_only_active': True}
NOT USING PCA
Loading val dataset
INITIALIZING CONNECTION
COPYING DATASET TABLE
CREATING TEMPORARY TABLE
NORMALIZING FEATURES
APPLYING PADDING STRATEGY
CLOSE CONNECTION
Not using keywords:  ['Bhis1d']
{'db_path': '../data/tmp/67de8e44-4dd3-4ebc-b993-2896b1c8dbe1.db', 'T': 24, 'L': 24, 'S': 0.2, 'B': 0, 'C': 0.4, 'MIN_FLARE_CLASS': 30, 'ALLOW_OVERLAPS': 1, 'padding_strategy': 'zero', 'keywords': 'de

100%|██████████| 27/27 [00:01<00:00, 19.00it/s]
2it [01:04, 32.39s/it]

...

100%|██████████| 24/24 [00:01<00:00, 19.03it/s]
8it [04:22, 32.96s/it]

Loading Datasets
----------------
Loading train dataset
INITIALIZING CONNECTION
COPYING DATASET TABLE
CREATING TEMPORARY TABLE
NORMALIZING FEATURES
APPLYING PADDING STRATEGY
CLOSE CONNECTION
Not using keywords:  ['Bhis1d']
{'db_path': '../data/tmp/e6e32b4f-957b-4eda-bedd-28f9243f7a3b.db', 'T': 24, 'L': 24, 'S': 0.2, 'B': 0, 'C': 0.4, 'MIN_FLARE_CLASS': 30, 'ALLOW_OVERLAPS': 1, 'padding_strategy': 'zero', 'keywords': 'default', 'not_keywords': ['Bhis1d'], 'use_PCA': False, 'n_PCA_components': 0, 'mode': 'train', 'splits': [0, 1, 2, 3, 4, 5, 6, 7], 'noise_level': 0.05, 'use_only_active': True}
NOT USING PCA
Loading val dataset
INITIALIZING CONNECTION
COPYING DATASET TABLE
CREATING TEMPORARY TABLE
NORMALIZING FEATURES
APPLYING PADDING STRATEGY
CLOSE CONNECTION
Not using keywords:  ['Bhis1d']
{'db_path': '../data/tmp/e6e32b4f-957b-4eda-bedd-28f9243f7a3b.db', 'T': 24, 'L': 24, 'S': 0.2, 'B': 0, 'C': 0.4, 'MIN_FLARE_CLASS': 30, 'ALLOW_OVERLAPS': 1, 'padding_strategy': 'zero', 'keywords': 'de

100%|██████████| 27/27 [00:01<00:00, 18.83it/s]
9it [04:54, 32.90s/it]

Loading Datasets
----------------
Loading train dataset
INITIALIZING CONNECTION
COPYING DATASET TABLE
CREATING TEMPORARY TABLE
NORMALIZING FEATURES
APPLYING PADDING STRATEGY
CLOSE CONNECTION
Not using keywords:  ['Bhis1d']
{'db_path': '../data/tmp/c2dd2c84-af37-4401-a2c8-08744bd2e928.db', 'T': 24, 'L': 24, 'S': 0.2, 'B': 0, 'C': 0.4, 'MIN_FLARE_CLASS': 30, 'ALLOW_OVERLAPS': 1, 'padding_strategy': 'zero', 'keywords': 'default', 'not_keywords': ['Bhis1d'], 'use_PCA': False, 'n_PCA_components': 0, 'mode': 'train', 'splits': [1, 2, 3, 4, 5, 6, 7, 8], 'noise_level': 0.05, 'use_only_active': True}
NOT USING PCA
Loading val dataset
INITIALIZING CONNECTION
COPYING DATASET TABLE
CREATING TEMPORARY TABLE
NORMALIZING FEATURES
APPLYING PADDING STRATEGY
CLOSE CONNECTION
Not using keywords:  ['Bhis1d']
{'db_path': '../data/tmp/c2dd2c84-af37-4401-a2c8-08744bd2e928.db', 'T': 24, 'L': 24, 'S': 0.2, 'B': 0, 'C': 0.4, 'MIN_FLARE_CLASS': 30, 'ALLOW_OVERLAPS': 1, 'padding_strategy': 'zero', 'keywords': 'de

100%|██████████| 26/26 [00:01<00:00, 17.89it/s]
10it [05:27, 32.72s/it]
